In [1]:
from transformers import AutoTokenizer
import transformers
import torch
import re

In [2]:
def get_sequences(prompt, tokenizer, llm_pipeline):
    sequences = llm_pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,
    )
    return sequences

def get_pipeline(model):
    pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    return pipeline

def get_response(gen_text):
    pattern = r'\[([^]]+)\]'
    matches = re.findall(pattern, gen_text)
    
    return matches

In [3]:
model_name = "/kaggle/input/llama-2/pytorch/7b-chat-hf/1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
llm_pipeline = get_pipeline(model_name)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [7]:
prompt = "Suggest 10 topics for the dialogue, where each suggestion is wrapped in square brackets."
response = get_sequences(prompt, tokenizer, llm_pipeline)
topics = response[0]["generated_text"].replace(prompt, "")
topics = get_response(topics)

In [10]:
d = {'question':[], 'answer':[]}
response = f"Let's discuss this topic: {topics[0]}?"
for i in range(6):
    if i%2 == 0:
        key = 'question'
    else:
        key = 'answer'
    prompt = f'''
        You are in a dialogue with your friend.
        Your friend says: "{response}"
        Continue the dialogue with your concise {key} in square brackets.
    '''
    seq = get_sequences(prompt, tokenizer, llm_pipeline)
    seq = seq[0]["generated_text"].replace(prompt, "")
    resp = get_response(seq)
    response = resp[0] if len(resp) > 0 else "Could you repeat?"
    d[key].append(response)